<a href="https://colab.research.google.com/github/20wiz/Common-Sense-Reasoning-ARC/blob/main/Common_Sense_Reasoning_on_ARC_(Challenge)_with_phi_1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())

2.5.1+cu121
True


T4 cpu 2 , CUDA available
V2-8  cpu 96 CUDA not available

In [4]:
try:
       import transformers
except ImportError:
    print("Transformers is not installed.")
else:
    print("Transformers is installed.")
    print(f"Transformers version: {transformers.__version__}")

Transformers is installed.
Transformers version: 4.47.1


In [8]:
try:
    import datasets
except ImportError:
    print("Datasets is not installed.")
else:
    print("Datasets is installed.")
    print(f"Datasets version: {datasets.__version__}")

Datasets is installed.
Datasets version: 3.2.0


In [ ]:
!pip install datasets


In [9]:
from datasets import load_dataset

# ARC Challenge 예시
arc_dataset = load_dataset('ai2_arc', 'ARC-Challenge')
# ARC Easy 예시
# arc_dataset = load_dataset('ai2_arc', 'ARC-Easy')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.00k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/190k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/204k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/55.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1119 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1172 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/299 [00:00<?, ? examples/s]

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "microsoft/phi-1_5"  # 실제 모델 경로/이름 확인 필요
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()


tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((2048,

추론 예제

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "microsoft/phi-1_5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

arc_dataset = load_dataset('ai2_arc', 'ARC-Challenge')

In [63]:

def make_prompt(question_stem, choices):
    """
    ARC 문제를 모델에 넣기 위한 단순 프롬프트 생성 예시
    """
    prompt = f"Question: {question_stem}\nChoices:\n"
    # for i, choice in choices.items():
    for text, label in zip(choices['text'], choices['label']):
        # print(label)
        # label = choice['label']  # 'A', 'B', 'C', 'D' 등
        # text = choice['text']
        prompt += f"{label}: {text}\n"
    prompt += "Answer:"
    return prompt

# 예시로 validation 셋의 앞 부분만 추론해보기
# num_samples = 3
num_samples = len(arc_dataset['validation'])
correct = 0
total = 0

samples = arc_dataset['validation'][:num_samples]
    # sample 구조 예시
    # {
    #   'id': [...],
    #   'question': ["문항1", "문항2", ...],
    #   'choices': [
    #        {"text": [...], "label": [...]},
    #        {"text": [...], "label": [...]}
    #   ],
    #   'answerKey': ["D", "C", ...]
    # }
questions = samples["question"]
choices_list = samples["choices"]
answers = samples["answerKey"]

# 질문 수(choices, answerKey와 길이 동일해야 함)
# 예: 2개의 질문이 있으면 range(2)로 loop
for q_idx in range(len(questions)):
  question_text = questions[q_idx]
  choice_dict = choices_list[q_idx]
  # print('choice_dict')
  # print(choice_dict)
  gold = answers[q_idx]  # 정답 레이블 (예: 'C')

  # 프롬프트 생성
  prompt = make_prompt(question_text, choice_dict)
  inputs = tokenizer(prompt, return_tensors='pt').to(device)

  # 모델 추론
  with torch.no_grad():
      outputs = model.generate(
          **inputs,
          max_new_tokens=1,   # 단일 토큰만 생성 (A/B/C/D)
          do_sample=False,    # Greedy decoding
      )

  # 모델 출력 디코딩
  output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  # 생성 결과에서 마지막 글자만 추출(A/B/C/D 중 하나일 것으로 가정)
  predicted_answer = output_text.strip()[-1]

  # 정답 비교
  total += 1
  if predicted_answer == gold:
      correct += 1

# 5. 전체 정확도 출력
print(f"Accuracy: {correct}/{total} = {correct/total*100:.2f}%")


choice_dict
{'text': ['Put the objects in groups.', 'Change the height of the ramp.', 'Choose different objects to roll.', 'Record the details of the investigation.'], 'label': ['A', 'B', 'C', 'D']}
choice_dict
{'text': ['fog', 'rain', 'drought', 'tornado'], 'label': ['A', 'B', 'C', 'D']}
choice_dict
{'text': ['The sun revolves around Earth.', 'Earth rotates around the sun.', 'The sun revolves on its axis.', 'Earth rotates on its axis.'], 'label': ['A', 'B', 'C', 'D']}
choice_dict
{'text': ['converting sunlight into electricity', 'looking for new coal reserves', 'finding reservoirs that contain oil', 'converting forests into farmland'], 'label': ['A', 'B', 'C', 'D']}
choice_dict
{'text': ['Shady areas increased.', 'Food sources increased.', 'Oxygen levels increased.', 'Available water increased.'], 'label': ['A', 'B', 'C', 'D']}
choice_dict
{'text': ['speed', 'a spotted coat', 'hunting strategies', 'claws that do not retract'], 'label': ['A', 'B', 'C', 'D']}
choice_dict
{'text': ['The 

In [18]:
type(arc_dataset['validation'][0])

dict

In [13]:
arc_dataset['validation'][0]

{'id': 'Mercury_SC_407695',
 'question': 'Juan and LaKeisha roll a few objects down a ramp. They want to see which object rolls the farthest. What should they do so they can repeat their investigation?',
 'choices': {'text': ['Put the objects in groups.',
   'Change the height of the ramp.',
   'Choose different objects to roll.',
   'Record the details of the investigation.'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'D'}

In [16]:
arc_dataset['validation'][0]['question']

'Juan and LaKeisha roll a few objects down a ramp. They want to see which object rolls the farthest. What should they do so they can repeat their investigation?'

In [37]:
# import json
# dataset = json.load(arc_dataset)
for sample in arc_dataset['validation'][:2].values():
# for sample in dataset['validation'][:2]:
  print (type(sample), sample, )

<class 'list'> ['Mercury_SC_407695', 'Mercury_7103565']
<class 'list'> ['Juan and LaKeisha roll a few objects down a ramp. They want to see which object rolls the farthest. What should they do so they can repeat their investigation?', 'High-pressure systems stop air from rising into the colder regions of the atmosphere where water can condense. What will most likely result if a high-pressure system remains in an area for a long period of time?']
<class 'list'> [{'text': ['Put the objects in groups.', 'Change the height of the ramp.', 'Choose different objects to roll.', 'Record the details of the investigation.'], 'label': ['A', 'B', 'C', 'D']}, {'text': ['fog', 'rain', 'drought', 'tornado'], 'label': ['A', 'B', 'C', 'D']}]
<class 'list'> ['D', 'C']


In [26]:
arc_dataset['validation']

Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 299
})

In [27]:
arc_dataset['validation'][:2]

{'id': ['Mercury_SC_407695', 'Mercury_7103565'],
 'question': ['Juan and LaKeisha roll a few objects down a ramp. They want to see which object rolls the farthest. What should they do so they can repeat their investigation?',
  'High-pressure systems stop air from rising into the colder regions of the atmosphere where water can condense. What will most likely result if a high-pressure system remains in an area for a long period of time?'],
 'choices': [{'text': ['Put the objects in groups.',
    'Change the height of the ramp.',
    'Choose different objects to roll.',
    'Record the details of the investigation.'],
   'label': ['A', 'B', 'C', 'D']},
  {'text': ['fog', 'rain', 'drought', 'tornado'],
   'label': ['A', 'B', 'C', 'D']}],
 'answerKey': ['D', 'C']}

In [33]:
arc_dataset['validation']['question'][:2]

['Juan and LaKeisha roll a few objects down a ramp. They want to see which object rolls the farthest. What should they do so they can repeat their investigation?',
 'High-pressure systems stop air from rising into the colder regions of the atmosphere where water can condense. What will most likely result if a high-pressure system remains in an area for a long period of time?']

In [34]:
arc_dataset['validation'][:2]['question']

['Juan and LaKeisha roll a few objects down a ramp. They want to see which object rolls the farthest. What should they do so they can repeat their investigation?',
 'High-pressure systems stop air from rising into the colder regions of the atmosphere where water can condense. What will most likely result if a high-pressure system remains in an area for a long period of time?']

In [36]:
s= arc_dataset['validation'][:2]
print(type(s))
s['question']

<class 'dict'>


['Juan and LaKeisha roll a few objects down a ramp. They want to see which object rolls the farthest. What should they do so they can repeat their investigation?',
 'High-pressure systems stop air from rising into the colder regions of the atmosphere where water can condense. What will most likely result if a high-pressure system remains in an area for a long period of time?']

In [40]:
s['choices']

[{'text': ['Put the objects in groups.',
   'Change the height of the ramp.',
   'Choose different objects to roll.',
   'Record the details of the investigation.'],
  'label': ['A', 'B', 'C', 'D']},
 {'text': ['fog', 'rain', 'drought', 'tornado'],
  'label': ['A', 'B', 'C', 'D']}]

In [42]:
s['choices'][0]['text']

['Put the objects in groups.',
 'Change the height of the ramp.',
 'Choose different objects to roll.',
 'Record the details of the investigation.']

In [43]:
s['choices'][0]['label']

['A', 'B', 'C', 'D']

In [52]:
c=s['choices'][0]
print(c)

{'text': ['Put the objects in groups.', 'Change the height of the ramp.', 'Choose different objects to roll.', 'Record the details of the investigation.'], 'label': ['A', 'B', 'C', 'D']}


In [57]:
for i, choice in c.items():
  print(i)
  print(choice)
  # print(choice['text'])

text
['Put the objects in groups.', 'Change the height of the ramp.', 'Choose different objects to roll.', 'Record the details of the investigation.']
label
['A', 'B', 'C', 'D']


In [62]:
len(arc_dataset['validation'])

299

In [60]:
arc_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'choices', 'answerKey'],
        num_rows: 1119
    })
    test: Dataset({
        features: ['id', 'question', 'choices', 'answerKey'],
        num_rows: 1172
    })
    validation: Dataset({
        features: ['id', 'question', 'choices', 'answerKey'],
        num_rows: 299
    })
})

In [64]:
arc_dataset['validation']['question'][1]

'High-pressure systems stop air from rising into the colder regions of the atmosphere where water can condense. What will most likely result if a high-pressure system remains in an area for a long period of time?'

In [65]:
arc_dataset['validation']['choices'][1]

{'text': ['fog', 'rain', 'drought', 'tornado'], 'label': ['A', 'B', 'C', 'D']}

In [67]:
arc_dataset['validation']['answerKey'][1]

'C'